In [4]:
import pandas as pd 
import numpy as np

In [5]:
set2 = pd.read_csv('lv2-2311.csv')

In [6]:
set2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
SCM_NO                1000 non-null int64
PRODUCT_SOLD          1000 non-null int64
REVENUE               1000 non-null float64
SHIPPING_TIME         1000 non-null int64
SHIPPING_COST         1000 non-null float64
PRODUCT_VOL           1000 non-null int64
MANUFACTURING_TIME    1000 non-null int64
MANUFACTURING_COST    1000 non-null float64
DEFECT_RATE           1000 non-null float64
PRODUCT_TYPE          1000 non-null object
ROUTE                 1000 non-null object
TOTAL_COST            1000 non-null float64
dtypes: float64(5), int64(5), object(2)
memory usage: 93.9+ KB


# 1

In [7]:
df1 = set2.copy()

In [8]:
p_list = df1['PRODUCT_TYPE'].unique().tolist()

In [9]:
res = pd.DataFrame(columns = df1.columns)
for p in p_list:
    tmp = df1[df1['PRODUCT_TYPE'] == p]
    q1_t = tmp['SHIPPING_TIME'].quantile(.25)
    q3_t = tmp['SHIPPING_TIME'].quantile(.75)
    iqr_t = q3_t - q1_t
    q1_c = tmp['SHIPPING_COST'].quantile(.25)
    q3_c = tmp['SHIPPING_COST'].quantile(.75)
    iqr_c = q3_c - q1_c

    tmp1 = tmp[(tmp['SHIPPING_TIME'] >= q1_t -  iqr_t) & (tmp['SHIPPING_TIME'] <= q3_t +iqr_t)]
    tmp2 = tmp1[(tmp1['SHIPPING_COST'] >= q1_c -  iqr_c) & (tmp1['SHIPPING_COST'] <= q3_c + iqr_c)]
    res = res.append(tmp2)
        

In [10]:
res['SHIPPING_TIME'] = res['SHIPPING_TIME'].astype(int)

A = res.groupby(['PRODUCT_TYPE'])['SHIPPING_COST'].mean().reset_index()

B = res.groupby(['PRODUCT_TYPE'])['SHIPPING_TIME'].mean().reset_index()

A['SHIPPING_TIME'] = B['SHIPPING_TIME']

A['RATIO'] = A['SHIPPING_COST'] / A['SHIPPING_TIME']


In [11]:
ans24 = A['RATIO'].min()

In [13]:
print('Q1 Result:', ans24)
print('Q1 Answer:', round(ans24,3))

Q1 Result: 0.9480963473110875
Q1 Answer: 0.948


# 2

In [14]:
df2 = set2.copy()

In [15]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [16]:
mm = MinMaxScaler()
sc = StandardScaler()

In [17]:
m_df = mm.fit_transform(df2[['PRODUCT_SOLD', 'REVENUE', 'PRODUCT_VOL','MANUFACTURING_TIME','MANUFACTURING_COST' ]])
sc_df = sc.fit_transform(df2[['PRODUCT_SOLD', 'REVENUE', 'PRODUCT_VOL','MANUFACTURING_TIME','MANUFACTURING_COST' ]])

In [18]:
from sklearn.cluster import KMeans

In [19]:
km1 = KMeans(init='random',random_state = 100, n_clusters=3, n_init = 100)
km2 = KMeans(init='random',random_state = 101, n_clusters=3, n_init = 100)

In [20]:
df2['sc_cluster'] = km1.fit(sc_df).predict(sc_df)
df2['m_cluster'] = km2.fit(m_df).predict(m_df)

In [21]:
s_agg = df2.groupby(['sc_cluster'])['SCM_NO'].count().reset_index()
s_agg['Max'] = s_agg['SCM_NO'].max()
smax = s_agg[s_agg['SCM_NO'] == s_agg['Max']]['sc_cluster'].values[0]
print(smax)

2


In [22]:
m_agg = df2.groupby(['m_cluster'])['SCM_NO'].count().reset_index()
m_agg['Max'] = m_agg['SCM_NO'].max()
mmax = m_agg[m_agg['SCM_NO'] == m_agg['Max']]['m_cluster'].values[0]
print(mmax)

1


In [23]:
tmp1 = df2[(df2['m_cluster'] == mmax) & (df2['sc_cluster'] == smax)]

In [24]:
A = len(tmp1[tmp1['PRODUCT_TYPE'] == 'PC'])/ len(tmp1)
ans25 = A

In [25]:
print('Q2 Result:', ans25)
print('Q2 Answer:', round(ans25,3))

Q2 Result: 0.4634146341463415
Q2 Answer: 0.463


# 3

In [26]:
df3 = set2.copy()

In [27]:
train = df3[df3['SCM_NO']% 5 != 0]
test = df3[df3['SCM_NO']% 5 == 0]

In [28]:
train_x = train[['PRODUCT_SOLD', 'SHIPPING_TIME', 'SHIPPING_COST', 'PRODUCT_VOL', 'MANUFACTURING_TIME', 'MANUFACTURING_COST']]
test_x = test[['PRODUCT_SOLD', 'SHIPPING_TIME', 'SHIPPING_COST', 'PRODUCT_VOL', 'MANUFACTURING_TIME', 'MANUFACTURING_COST']]

In [29]:
train_y = train[['TOTAL_COST']]
test_y = test[['TOTAL_COST']]

In [30]:
from statsmodels.api import add_constant, OLS
train_x_const = add_constant(train_x)
model = OLS(train_y, train_x_const).fit()
model.summary()

c:\dstest\python374\lib\site-packages\numpy\core\fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             TOTAL_COST   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     2.388
Date:                Tue, 21 Nov 2023   Prob (F-statistic):             0.0271
Time:                        15:10:34   Log-Likelihood:                -5575.9
No. Observations:                 800   AIC:                         1.117e+04
Df Residuals:                     793   BIC:                         1.120e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                583.9214     41.470     14.080      0.000     502.517     665.326
PRODUCT_SOLD          -0.0430      0.031     -1.397      0.163      -0.103       0.017
SHIPPING_TIME          0.2932      3.207      0.091      0.927      -6.001       6.588
SHIPPING_COST          6.0453      3.279      1.844      0.066      -0.390      12.481
PRODUCT_VOL           -0.0605      0.037     -1.655      0.098      -0.132       0.011
MANUFACTURING_TIME     0.4055      1.078      0.376      0.707      -1.711       2.522
MANUFACTURING_COST    -0.6942      0.331     -2.095      0.037      -1.345      -0.044
==============================================================================
Omnibus:                      250.829   Durbin-Watson:                   2.071
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               38.547
Skew:                           0.015   Prob(JB):                     4.26e-09
Kurtosis:                       1.925   Cond. No.                     3.64e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.64e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [31]:
conf = model.conf_int(alpha=0.05).reset_index()
C = conf[conf['index']=='SHIPPING_COST'][0].values[0]
D = conf[conf['index']=='SHIPPING_COST'][1].values[0]
conf[conf['index']=='SHIPPING_COST']

,index,0,1
3,SHIPPING_COST,-0.390491,12.481131


In [32]:
test1 = test[test['SCM_NO'] == 870]

In [33]:
test_x1 = test1[['PRODUCT_SOLD', 'SHIPPING_TIME', 'SHIPPING_COST', 'PRODUCT_VOL', 'MANUFACTURING_TIME', 'MANUFACTURING_COST']]

In [34]:
E = model.predict(add_constant(test_x))[test1.index.tolist()[0]]

In [35]:
ans26 = abs(C) + abs(D) + abs(E)

In [36]:
print('Q3 Result:', ans26)
print('Q3 Answer:', round(ans26,1))

Q3 Result: 580.3946496852564
Q3 Answer: 580.4


### Summary

In [37]:
print('Q1 Result:', ans24)
print('Q1 Answer:', round(ans24,3))
print('Q2 Result:', ans25)
print('Q2 Answer:', round(ans25,3))
print('Q3 Result:', ans26)
print('Q3 Answer:', round(ans26,1))

Q1 Result: 0.9480963473110875
Q1 Answer: 0.948
Q2 Result: 0.4634146341463415
Q2 Answer: 0.463
Q3 Result: 580.3946496852564
Q3 Answer: 580.4
